In [1]:
from transformers import pipeline

protgpt2 = pipeline('text-generation', model="nferruz/ProtGPT2")

2023-02-28 05:09:10.571734: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import torch

prompt = "<|endoftext|>"

args = dict(
  max_length=512,
  do_sample=True,
  top_k=950,
  repetition_penalty=1.2,
  num_return_sequences=10,
  eos_token_id=0
)

torch.cuda.empty_cache()
sequences = protgpt2(prompt, **args)

/home/kiddos/.local/lib/python3.8/site-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [3]:
def filter_sequences(result):
  output = []
  for i, entry in enumerate(sequences):
    seq = entry['generated_text']
    seq = ''.join(seq[15:].split('\n'))
    if 'X' in seq:
      continue
    output.append(seq)
  return output


filter_sequences(sequences)

['VADSVTVTGAILSDFDEDGSNEIVVASPTGEVYAYLNASVLSFTFSDNDIHIALTDGGSGTAIATRQHSGFIDGGTINYRTCSGTASLFLDYDSISGTIPAGETLATIDVTVIDDLSDEAEDTLSLTIAGITCSDCSTVCSLTIEDDGLYSSYDDDSSVYETYTFYEDQHFQAHYFSGYTDTGSILYMGTCSGSADFRFYYNSSLGTVSGGDWQSYTATATTSADTELEIYVNSNTGDSQEWCCTFSITESTVVDYTVTDANSIADGDLFIGQDCNYSIQTYTGTYNSGQTLFMDTCTGGADYRFYYDSSVGVVSGGVWAAYSATGASDTTDTIDIGINGNVSDTEEWCCTINLTESATVDYRIAEENTAADGDTAIGQDLTFTTTLDYGTHCVGSWTGAYNTGAKLYMDTCSGGTDFRFYVDSSFSGTSGTDWHSYSATATSDGADVDIKIDSNSDDTKEYCCTLTLTEHGHVVSSWQLYAVFHYTLVDLDAATFLKWNGDELYHDGDAVATDELGVANISMEVRDHHDGSGIGTKTQLKAHGTYEVFSFYSSEHDAGGMSIINGEKSWGKVVYGSSPHPVPIIEDHNPYDESRWAAWGLYRNGSMFIQIHPEGQWYDSYPAGEWFTTYPGYEKHEDMATLSANSWDEYIFNYDITGIEDGADTIYVSYKAVDGYGSHSTAATVQIQNVTRPEVYLIDFTDEEELSGLEVYADDAPMYYGTQWDDPSSRYQVTFDKSGELRIRIDNPGWEWNSLDVISIERISDSDHFVQTVEDSIPLIKRILRMKTLLPWALEPIYGASIVIFPVMGVVLLQRRIDRTERWATLKRQIRNILPFRPTTPEIPSTGVRSRQVARIRAVLRRDKPVSSARVASRCPSRRPQYVQQRRWRRRVNPRTARSRWRPSPWSYSPRHRGSRTEWTARRWSPASPRLCLPQPIRRTTRSRTSRTRCFAASRRS',
 'SGRARRQTSSVRILLVEDNRDAAESLGDVLEARECI

In [4]:
generated = filter_sequences(sequences)

In [5]:
generated[0]

'VADSVTVTGAILSDFDEDGSNEIVVASPTGEVYAYLNASVLSFTFSDNDIHIALTDGGSGTAIATRQHSGFIDGGTINYRTCSGTASLFLDYDSISGTIPAGETLATIDVTVIDDLSDEAEDTLSLTIAGITCSDCSTVCSLTIEDDGLYSSYDDDSSVYETYTFYEDQHFQAHYFSGYTDTGSILYMGTCSGSADFRFYYNSSLGTVSGGDWQSYTATATTSADTELEIYVNSNTGDSQEWCCTFSITESTVVDYTVTDANSIADGDLFIGQDCNYSIQTYTGTYNSGQTLFMDTCTGGADYRFYYDSSVGVVSGGVWAAYSATGASDTTDTIDIGINGNVSDTEEWCCTINLTESATVDYRIAEENTAADGDTAIGQDLTFTTTLDYGTHCVGSWTGAYNTGAKLYMDTCSGGTDFRFYVDSSFSGTSGTDWHSYSATATSDGADVDIKIDSNSDDTKEYCCTLTLTEHGHVVSSWQLYAVFHYTLVDLDAATFLKWNGDELYHDGDAVATDELGVANISMEVRDHHDGSGIGTKTQLKAHGTYEVFSFYSSEHDAGGMSIINGEKSWGKVVYGSSPHPVPIIEDHNPYDESRWAAWGLYRNGSMFIQIHPEGQWYDSYPAGEWFTTYPGYEKHEDMATLSANSWDEYIFNYDITGIEDGADTIYVSYKAVDGYGSHSTAATVQIQNVTRPEVYLIDFTDEEELSGLEVYADDAPMYYGTQWDDPSSRYQVTFDKSGELRIRIDNPGWEWNSLDVISIERISDSDHFVQTVEDSIPLIKRILRMKTLLPWALEPIYGASIVIFPVMGVVLLQRRIDRTERWATLKRQIRNILPFRPTTPEIPSTGVRSRQVARIRAVLRRDKPVSSARVASRCPSRRPQYVQQRRWRRRVNPRTARSRWRPSPWSYSPRHRGSRTEWTARRWSPASPRLCLPQPIRRTTRSRTSRTRCFAASRRS'

In [6]:
from transformers import AutoTokenizer, EsmForProteinFolding

tokenizer = AutoTokenizer.from_pretrained("facebook/esmfold_v1")
model = EsmForProteinFolding.from_pretrained("facebook/esmfold_v1")

Some weights of EsmForProteinFolding were not initialized from the model checkpoint at facebook/esmfold_v1 and are newly initialized: ['esm.contact_head.regression.weight', 'esm.contact_head.regression.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model.esm = model.esm.float()
model.trunk.set_chunk_size(64)

In [8]:
import torch

test_protein = generated[0]

tokenized_input = tokenizer([test_protein], return_tensors="pt", add_special_tokens=False)

with torch.no_grad():
  output = model(tokenized_input['input_ids'])

In [9]:
from transformers.models.esm.openfold_utils.protein import to_pdb, Protein as OFProtein
from transformers.models.esm.openfold_utils.feats import atom14_to_atom37

def convert_outputs_to_pdb(outputs):
  final_atom_positions = atom14_to_atom37(outputs["positions"][-1], outputs)
  outputs = {k: v.to("cpu").numpy() for k, v in outputs.items()}
  final_atom_positions = final_atom_positions.cpu().numpy()
  final_atom_mask = outputs["atom37_atom_exists"]
  pdbs = []
  for i in range(outputs["aatype"].shape[0]):
    aa = outputs["aatype"][i]
    pred_pos = final_atom_positions[i]
    mask = final_atom_mask[i]
    resid = outputs["residue_index"][i] + 1
    pred = OFProtein(
      aatype=aa,
      atom_positions=pred_pos,
      atom_mask=mask,
      residue_index=resid,
      b_factors=outputs["plddt"][i],
      chain_index=outputs["chain_index"][i] if "chain_index" in outputs else None,
    )
    pdbs.append(to_pdb(pred))
  return pdbs


pdb = convert_outputs_to_pdb(output)

In [12]:
import py3Dmol

view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js', width=800, height=400)
view.addModel("".join(pdb), 'pdb')
view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [13]:
with open('test.pdb', 'w') as f:
  f.write('\n'.join(pdb))